# Gym basic
## Environment



In [6]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[-0.0167726  -0.04001986  0.04177399  0.03153886]
[-0.017573   -0.2357152   0.04240477  0.33710375]
[-0.0222873  -0.43141413  0.04914685  0.64285153]
[-0.03091558 -0.23701042  0.06200387  0.36604142]
[-0.03565579 -0.04282187  0.0693247   0.0935358 ]
[-0.03651223  0.15124151  0.07119542 -0.17649491]
[-0.0334874  -0.04482329  0.06766552  0.13777101]
[-0.03438386  0.14926752  0.07042094 -0.13282049]
[-0.03139851 -0.04678879  0.06776453  0.18122175]
[-0.03233429  0.14730135  0.07138897 -0.08933773]
[-0.02938826 -0.04876741  0.06960221  0.22498682]
[-0.03036361  0.14529432  0.07410195 -0.04495443]
[-0.02745772 -0.05080768  0.07320286  0.27015737]
[-0.02847388 -0.24689376  0.07860601  0.58500123]
[-0.03341175 -0.05295579  0.09030603  0.31807888]
[-0.03447087 -0.24924012  0.09666761  0.6378195 ]
[-0.03945567 -0.44556767  0.109424    0.959312  ]
[-0.04836702 -0.641977    0.12861024  1.2842705 ]
[-0.06120656 -0.83848023  0.15429565  1.6142982 ]
[-0.07797617 -1.0350496   0.18658161  1.9508326 ]


In [7]:
env = gym.make('CartPole-v0')
print(env.action_space)
#> Discrete(2)
print(env.observation_space)
#> Box(4,)

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


## Environment Creation
### Subclassing gym.Env
Create the "GridWorldEnv"
1. Consider the environment: a grid world, fixed size during creation
2. Observations provide the location
3. 4 actions in the environment
4. Done signal is issued when target cell grid is located
5. Rewards are binary and sparse, immediate reward is always zero, unless the agent has reached the target, then it is 1.

# Using stable-baselines3
https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html

In [8]:
import gym

from stable_baselines3 import A2C

env = gym.make('CartPole-v1')

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

F:\Anaconda3\envs\rl_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
F:\Anaconda3\envs\rl_env\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27.3     |
|    ep_rew_mean        | 27.3     |
| time/                 |          |
|    fps                | 787      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.683   |
|    explained_variance | -0.0422  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.581    |
|    value_loss         | 15.6     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 26.4     |
|    ep_rew_mean        | 26.4     |
| time/                 |          |
|    fps                | 854      |
|    iterations         | 200      |
|    time_elapsed 

# RL unino API reference

## API-伪代码
1. 初始化环境和Agent
2. 对于每个回合，Agent选择动作
3. 环境接收动作并反馈下一个状态和奖励
4. 智能体进行策略更新（Learning process）
5. 多个回合后算法收敛，保存模型，后续分析，作图

# DQN
基于深度学习的Qlearning，结合价值函数近似与神经网络
- 采用目标网络和经验回放进行网络训练

## 深度Q网络基本接口
- 初始化缓冲区D，容量为N
- 初始化状态-动作函数，即带有权重 $\theta$ 的Q网络 
- 初始化状态-动作函数，即带有权重 $\theta$' 的Q网络 （目标网络）
- 执行M回合，对于每个回合
    - 初始化环境，得到初始状态s1
    - 循环T个时间步长，对于每个时间步t
        - 使用e-greedy策略选择动作
        - 环境根据at 反馈奖励rt和下一个状态st+1
        - 环境更新状态st = st+1
        - 储存转移，(st, at, st+1, rt) 到经验回放D中
        - 更新策略如下：
            1. 从D中随机采样一个小批量的转移
            2. 计算实际Q值，
            3. 对于损失函数关于参数 $\theta$ 做随机梯度下降
            4. 没C步重置Q' = Q （更新目标网络）

### 解决CartPole-v0

In [3]:
import gym

env = gym.make('CartPole-v0')
env.seed(1)
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n
print(f"状态数{n_states}, 动作数{n_actions}")
# 状态：车的位置，车的速度，杆角度，杆顶部速度
# 动作：向左，向右
state = env.reset()
print(state)

状态数4, 动作数2
[ 0.03073904  0.00145001 -0.03088818 -0.03131252]


# Test for tensorboard

In [ ]:
import tensorflow as tf
import datetime
mnist = tf.keras.datasets.mnist

# rm -rf ./logs/

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "test_tf_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

